This notebook is intended to aid the construction of a first pointing model with the Auxiliary Telescope. 

At this early stages, and given the small field-of-view of the telescope, it is really hard to get a start on the field. To help on the process, this notebook implements a Gridding routine. The routine will scan around the current position of the telescope and waits for the user to specify if is should continue, go to the next or previous pointing or stop. 



In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

## Disable auto download of iers data. 

The main reason for disabling this is that the containers on the control network do not have access to the internet. Trying to download data when there is no outside access means a long time before the connection times out. 

In [ ]:
from astropy.utils import iers
iers.conf.auto_download = False

## Check value of the `LSST_DDS_DOMAIN` variable.

For the AT early work at the summit, the expected value for `LSST_DDS_DOMAIN=lsatmcs`

In [ ]:
import os
print(os.environ["LSST_DDS_DOMAIN"])

## Create a domain and remotes

The ATTCS class uses the following remotes:

  - atmcs 
  - atptg
  - ataos
  - atpneumatics 
  - athexapod
  - atdome
  - atdometrajectory


In [ ]:
d = salobj.Domain()

In case, you need to build the idl files, copy the following command to a cell and run it. 

```
%%script bash 
make_idl_files.py ATMCS ATPtg ATAOS ATPneumatics ATHexapod ATDome ATDomeTrajectory
```

In [ ]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")

In [ ]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task)

In [ ]:
ss = await atptg.evt_summaryState.next(flush=False, timeout=1.)
print(salobj.State(ss.summaryState))

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

In [ ]:
ss = await atptg.evt_summaryState.next(flush=False, timeout=1.)
print(salobj.State(ss.summaryState))

# Slew and Track an Alt/Az position

While the pointing component does not support the slew and track of an alt/az position, the following cells will provide a quick and easy way to perform this task. The user specified a position in Alt/Az and it will use astropy coordinate library to convert it to RA/Dec. 

For that we will need the location of the observatory and the time. Location is defined as an astropy `EarthLocation` and time is taken from the pointing component and then creating an astropy `Time` object.

Obervatory location (lon, lat and height extracted from: https://github.com/lsst/sims_utils)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
alt = 80. * u.deg
az = 0. * u.deg

The next cell will convert the specified Alt/Az into RA/Dec coordinates in ICRS. This coordinate will be used to slew and track. 

In [ ]:
await asyncio.sleep(1.)
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

await atptg.cmd_raDecTarget.set_start(
    targetName="local",
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt.value,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print(f"raDecTarget ra={cmd_radec.ra!r} hour; "
      f"declination={cmd_radec.dec!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)



In [ ]:
await asyncio.gather(atmcs.close(), 
                    atptg.close())

await d.close()

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10.)

In [ ]:
await atptg.cmd_pointLoadModel.set_start(pointingFile='/home/saluser/repos/ts_pointing_common/install/data/at.mod')

In [ ]:
await atptg.cmd_pointLoadModel.set_start(pointingFile='auxtel_20190911.dat')

In [ ]:
await asyncio.sleep(5.)

for i in range(10):
    time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
    time_err = curr_time_atptg - Time.now()
    print(f"Time error={time_err.sec:0.2f} sec")

    # Compute RA/Dec for commanded az/el
    cmd_elaz = AltAz(alt=alt, az=az, 
                     obstime=curr_time_atptg.tai, 
                     location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS)

    await atptg.cmd_raDecTarget.set_start(
        targetName="local",
        targetInstance=ATPtg.TargetInstances.CURRENT,
        frame=ATPtg.CoordFrame.ICRS,
        epoch=2000,  # should be ignored: no parallax or proper motion
        equinox=2000,  # should be ignored for ICRS
        ra=cmd_radec.ra.hour,
        declination=cmd_radec.dec.deg,
        parallax=0,
        pmRA=0,
        pmDec=0,
        rv=0,
        dRA=0,
        dDec=0,
        rotPA=180.-alt.value,
        rotFrame=ATPtg.RotFrame.FIXED,
        rotMode=ATPtg.RotMode.FIELD,
        timeout=10
    )

    print(f"run: {i+1}")
    print(f"raDecTarget ra={cmd_radec.ra!r} hour; "
          f"declination={cmd_radec.dec!r} deg")
    await asyncio.sleep(1.)
    # script.ataos.cmd_enableCorrection.set(hexapod=True)
    # await script.ataos.cmd_enableCorrection.start(timeout=10)

await atptg.cmd_stopTracking.start(timeout=10.)


In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=80.)

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10.)

In [ ]:
await asyncio.sleep(5)
model = await atptg.evt_pointingModel.next(flush=False, timeout=1)

In [ ]:
model = atptg.evt_pointingModel.get()

In [ ]:
model.pointingModelTermNames, model.pointingModelTermValues